## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [ ]:
#install package
! pip install python-dotenv

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")
os.environ["DASHSCOPE_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")
os.environ["ZHIPUAI_API_KEY"] = os.getenv("ZHIPUAI_API_KEY")
zhipukey =  os.getenv("ZHIPUAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
! pip install langchain
! pip install --upgrade httpx httpx-sse PyJWT
! pip install langchainhub
! pip install openai
! pip install langchain_openai
! pip install langchain_community
! pip install pyjwt

In [3]:
! pip install --upgrade httpx httpx-sse PyJWT


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
! pip show pydantic
! pip show langchain-core
! pip show langchain_openai

### 使用langchain 0.2 大版本 接入智谱
### use langchain 0.2 version with ChatZhipuAI
*****

In [4]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="你的角色是一个诗人."),
    HumanMessage(content="用七言绝句的形式写一首关于AI的诗."),
]

response = chat.invoke(messages)
print(response.content)

ImportError: jwt package not found, please install it with`pip install pyjwt`

- Steaming output 流式输出

In [ ]:
#流式
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

streaming_chat = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
    streaming=True,
    callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),
)
streaming_chat.invoke(messages)


- Chain 链式调用

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

prompt_template = "给我讲一个关于{adjective}的笑话"
prompt = PromptTemplate(input_variables=["adjective"],
                        template=prompt_template)
tongyi_joke_chain = LLMChain(llm=chat, prompt=prompt)
res = tongyi_joke_chain.invoke("程序员")
print(res)

- Chain of LCEL

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")
output_parser = StrOutputParser()

chain = prompt | chat | output_parser

res = chain.invoke({"topic": "企鹅"})
print(res)

- Agent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool


#自定义工具
@tool
def search(query: str):
    """Call to surf the web."""
    # 返回假数据
    return ["天气非常好万里无云"]


tools = [search]
#提示词
prompt = hub.pull("hwchase17/react")
# Construct the ReAct agent
agent = create_react_agent(chat, tools, prompt)
#执行
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True)
agent_executor.invoke({"input": "北京今天的天气如何?"})

- Agents with TAVILY

In [ ]:
from langchain_community.chat_models import ChatZhipuAI
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatZhipuAI(temperature=0.01, model="glm-4")
tools = [TavilySearchResults(max_results=2)]
#prompt = hub.pull("hwchase17/react")
#print(prompt)
#这里可以自定义中文提示词
system = """
请尽可能回答下列问题。您可以使用以下工具:

{tools}

请使用以下格式:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

开始!

Question: {input}

Thought:{agent_scratchpad}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{input}"),
])
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True)
agent_executor.invoke({"input": "黄龙江一派全都带蓝牙是什么意思?"})